# Neeps - Resit

In [1]:
import findspark
import pandas as pd
findspark.init()

SVR = '192.168.31.31'
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

sc = (SparkSession.builder.appName('app15-4') 
      .master(f'spark://{SVR}:7077') 
      .config('spark.sql.warehouse.dir', f'hdfs://{SVR}:9000/user/hive/warehouse') 
      .config('spark.cores.max', '4') 
      .config('spark.executor.instances', '1') 
      .config('spark.executor.cores', '2') 
      .config('spark.executor.memory', '10g') 
      .enableHiveSupport().getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
ut_staff = sc.read.table('sqlzoo.ut_staff')
ut_student = sc.read.table('sqlzoo.ut_student')
ut_event = sc.read.table('sqlzoo.ut_event')
ut_room = sc.read.table('sqlzoo.ut_room')
ut_attends = sc.read.table('sqlzoo.ut_attends')
ut_teaches = sc.read.table('sqlzoo.ut_teaches')
ut_occurs = sc.read.table('sqlzoo.ut_occurs')
ut_modle = sc.read.table('sqlzoo.ut_modle')
ut_week = sc.read.table('sqlzoo.ut_week')

## 1.
**Give the day and the time of the event co72002.L01.**

In [3]:
(ut_event.filter(ut_event['id']=='co72002.L01')
 .select('dow', 'tod')
 .toPandas())

,dow,tod
0,Monday,10:00


## 2.
**For each event in module co72003 show the day, the time and the place.**

In [4]:
(ut_event.filter(ut_event['modle']=='co72003')
 .select('id', 'dow', 'tod', 'room')
 .toPandas())

,id,dow,tod,room
0,co72003.L01,Monday,14:00,cr.B12


## 3.
**List the id of the events taught by 'Chisholm, Ken'.**

In [5]:
(ut_event.withColumnRenamed('id', 'event')
 .join(ut_teaches, on='event')
 .join(ut_staff.filter(ut_staff['name']=='Chisholm, Ken')
        .withColumnRenamed('id', 'staff'), on='staff')
 .select('event')
 .toPandas())

,event
0,co22006.L01
1,co22006.L02
2,co22006.L03
3,co22006.T01
4,co22006.T02
5,co72013.L01
6,co72013.L02
7,co72013.T01
8,co72013.T02
9,co72013.T06


## 4.
**List the staff who teach in cr.SMH.**

In [6]:
(ut_staff.withColumnRenamed('id', 'staff')
 .join(ut_teaches, on='staff')
 .join(ut_event.filter(ut_event['room']=='cr.SMH')
        .withColumnRenamed('id', 'event'), on='event')
 .select('name')
 .distinct()
 .orderBy('name')
 .toPandas())

,name
0,"Barclay, Ken"
1,"Chisholm, Ken"
2,"Cumming, Andrew"
3,"Greig, Frank"
4,"Hastie, Colin"
5,"Jackson, Jim"
6,"Kemmer, Rob"
7,"Lawson, Alistair"
8,"Macaulay, Catriona"
9,"Maclean, Aileen"


## 5.
**Show the total number of hours (over the whole semester) of classes for com.IS.a**

In [7]:
(ut_event.withColumnRenamed('id', 'event')
 .join(ut_occurs, on='event')
 .join(ut_attends.filter(col('student')=='com.IS.a'), on='event')
 .groupBy()
 .agg(sum('duration').alias('hrs'))
 .toPandas())

,hrs
0,60


In [8]:
sc.stop()